In [129]:
import numpy as np 
import time 
from useful_function_joint import *
import pymanopt
from pymanopt import manifolds, optimizers, tools, core
from pymanopt.core.problem  import Problem
from typing import Sequence

import networkx as nx

In [428]:
scale_shift = 50

In [429]:
e1 = np.array((1,0,0))*scale_shift
e2 = np.array((0,1,0))*scale_shift
e3 = np.array((0,0,1))*scale_shift
e0 = np.array([0,0,0])

In [430]:
t1 = np.array((25,0,0))
t2 = np.array((560,0,0))
t3 = np.array((515,0,25))
t4 = np.array((0,0,0))
t5 = np.array((90,0,0))

In [431]:
joints_ref = np.array([0,-90,90,0,0,0])*np.pi/180

In [432]:
R1 = Rz(joints_ref[0])
R2 = Ry(joints_ref[1])
R3 = Ry(joints_ref[2])
R4 = Rx(joints_ref[3])
R5 = Ry(joints_ref[4])
R6 = Rx(joints_ref[5])

In [433]:
T1 = T(t1)
T2 = T(t2)
T3 = T(t3)
T4 = T(t4)
T5 = T(t5)

In [434]:
axes = ['z','y','y','x','y','x']
translation = [T(e0),T1,T2,T3,T4,T5]
rotation = [R1,R2,R3,R4,R5,R6]

step_vect = {'z': [e0,e3],
             'y': [e2,-e2],
             'x': [e0,-e1]}

In [458]:
position = np.array([400,0,695])#pose_robot_(joints_ref,'camera')[:3,3]
#np.array([400,0,695])

G = nx.Graph()

G.add_node(1,pos_ref=e1)
G.add_node(2,pos_ref=e2)
G.add_edge(1,2, weight  = np.linalg.norm(e1-e2))


count = 2

for i, ax in enumerate(axes):
    v1,v2 = step_vect[ax]

    transform = []

    for j in range(i+1):
        transform.append(translation[j])
        transform.append(rotation[j])
    transform = np.array(transform)
    transformation = np.linalg.multi_dot(transform)

    G.add_node(count+1,pos_ref=(transformation@T(v2))[:3,3])
    G.add_node(count+2,pos_ref=(transformation@T(v1))[:3,3])

    node_pos_ref = nx.get_node_attributes(G,'pos_ref')

    G.add_edge(count+2,count+1, weight = np.linalg.norm(node_pos_ref[count+1]-node_pos_ref[count+2]))

    G.add_edge(count,count+1, weight = np.linalg.norm(node_pos_ref[count+1]-node_pos_ref[count]))
    G.add_edge(count-1,count+1, weight = np.linalg.norm(node_pos_ref[count+1]-node_pos_ref[count-1]))

    G.add_edge(count,count+2, weight = np.linalg.norm(node_pos_ref[count+2]-node_pos_ref[count]))
    G.add_edge(count-1,count+2, weight = np.linalg.norm(node_pos_ref[count+2]-node_pos_ref[count-1]))

    count = count + 2

mapping = {1: "x", 2: "y", 3: "a1_plus", 4: "a1_origin", 5: "a2_minus", 6: " a2_plus", 7: "a3_minus", 8:"a3_plus", 
           9: "a4_minus", 10:"a4_origin", 11: "a5_minus", 12:"a5_plus", 13: "a6_minus", 14:"a6_origin"}
G = nx.relabel_nodes(G, mapping)

G.add_edge('a1_origin','a6_origin', weight = np.linalg.norm(position))

G.add_edge('x','a6_origin', weight = np.linalg.norm(position-e1))
G.add_edge('y','a6_origin', weight = np.linalg.norm(position-e2))
G.add_edge('a1_plus','a6_origin', weight = np.linalg.norm(position-e3))
  


In [459]:
G.edges

EdgeView([('x', 'y'), ('x', 'a1_plus'), ('x', 'a1_origin'), ('x', 'a6_origin'), ('y', 'a1_plus'), ('y', 'a1_origin'), ('y', 'a6_origin'), ('a1_plus', 'a1_origin'), ('a1_plus', 'a2_minus'), ('a1_plus', ' a2_plus'), ('a1_plus', 'a6_origin'), ('a1_origin', 'a2_minus'), ('a1_origin', ' a2_plus'), ('a1_origin', 'a6_origin'), ('a2_minus', ' a2_plus'), ('a2_minus', 'a3_minus'), ('a2_minus', 'a3_plus'), (' a2_plus', 'a3_minus'), (' a2_plus', 'a3_plus'), ('a3_minus', 'a3_plus'), ('a3_minus', 'a4_minus'), ('a3_minus', 'a4_origin'), ('a3_plus', 'a4_minus'), ('a3_plus', 'a4_origin'), ('a4_minus', 'a4_origin'), ('a4_minus', 'a5_minus'), ('a4_minus', 'a5_plus'), ('a4_origin', 'a5_minus'), ('a4_origin', 'a5_plus'), ('a5_minus', 'a5_plus'), ('a5_minus', 'a6_minus'), ('a5_minus', 'a6_origin'), ('a5_plus', 'a6_minus'), ('a5_plus', 'a6_origin'), ('a6_minus', 'a6_origin')])

In [460]:
nx.get_node_attributes(G,'pos_ref')

{'x': array([50,  0,  0]),
 'y': array([ 0, 50,  0]),
 'a1_plus': array([ 0.,  0., 50.]),
 'a1_origin': array([0., 0., 0.]),
 'a2_minus': array([ 25., -50.,   0.]),
 ' a2_plus': array([25., 50.,  0.]),
 'a3_minus': array([ 25., -50., 560.]),
 'a3_plus': array([ 25.,  50., 560.]),
 'a4_minus': array([490.,   0., 585.]),
 'a4_origin': array([540.,   0., 585.]),
 'a5_minus': array([540., -50., 585.]),
 'a5_plus': array([540.,  50., 585.]),
 'a6_minus': array([580.,   0., 585.]),
 'a6_origin': array([630.,   0., 585.])}

In [461]:
Y_init = np.zeros((3,35))

for i, e in enumerate(list(G.edges)):
    p1 = G.nodes[e[0]]['pos_ref']
    p2 = G.nodes[e[1]]['pos_ref']
    Y_init[:,i] = (p2-p1)/max(np.linalg.norm(p2-p1),10**-9)

Y_init[:3,11]

array([ 0.4472136 , -0.89442719,  0.        ])

In [462]:
cost(Y_init)

1.6616468201391399e-09

In [463]:
nx.get_node_attributes(G,'pos_ref')


{'x': array([50,  0,  0]),
 'y': array([ 0, 50,  0]),
 'a1_plus': array([ 0.,  0., 50.]),
 'a1_origin': array([0., 0., 0.]),
 'a2_minus': array([ 25., -50.,   0.]),
 ' a2_plus': array([25., 50.,  0.]),
 'a3_minus': array([ 25., -50., 560.]),
 'a3_plus': array([ 25.,  50., 560.]),
 'a4_minus': array([490.,   0., 585.]),
 'a4_origin': array([540.,   0., 585.]),
 'a5_minus': array([540., -50., 585.]),
 'a5_plus': array([540.,  50., 585.]),
 'a6_minus': array([580.,   0., 585.]),
 'a6_origin': array([630.,   0., 585.])}

In [464]:
G.edges()
weight

{('x', 'y'): 70.71067811865476,
 ('x', 'a1_plus'): 70.71067811865476,
 ('x', 'a1_origin'): 50.0,
 ('x', 'a6_origin'): 823.786987030021,
 ('y', 'a1_plus'): 70.71067811865476,
 ('y', 'a1_origin'): 50.0,
 ('y', 'a6_origin'): 861.1765208132418,
 ('a1_plus', 'a1_origin'): 50.0,
 ('a1_plus', 'a2_minus'): 75.0,
 ('a1_plus', ' a2_plus'): 75.0,
 ('a1_plus', 'a6_origin'): 826.5137627408269,
 ('a1_origin', 'a2_minus'): 55.90169943749474,
 ('a1_origin', ' a2_plus'): 55.90169943749474,
 ('a1_origin', 'a6_origin'): 859.723792854426,
 ('a2_minus', ' a2_plus'): 100.0,
 ('a2_minus', 'a3_minus'): 560.0,
 ('a2_minus', 'a3_plus'): 568.8585061331157,
 (' a2_plus', 'a3_minus'): 568.8585061331157,
 (' a2_plus', 'a3_plus'): 560.0,
 ('a3_minus', 'a3_plus'): 100.0,
 ('a3_minus', 'a4_minus'): 468.3481610938597,
 ('a3_minus', 'a4_origin'): 518.0250959171766,
 ('a3_plus', 'a4_minus'): 468.3481610938597,
 ('a3_plus', 'a4_origin'): 518.0250959171766,
 ('a4_minus', 'a4_origin'): 50.0,
 ('a4_minus', 'a5_minus'): 70.71

In [468]:

pos_ref = nx.get_node_attributes(G,'pos_ref')

weight = nx.get_edge_attributes(G, "weight")
D = np.diag([weight[i] for i in weight.keys()])
C = incidence_matrix_(G,oriented=True).toarray()
C = np.array(C)
print(C.shape)
d =3
n = D.shape[0]

manifold_ = manifolds.Oblique(d,n)

Q2 = -D@(C.transpose()@np.linalg.pinv(C@C.transpose()))@C@D;
Q1 = D@D;

@pymanopt.function.numpy(manifold_)
def cost(Y_):
    return np.trace(Q1 + Q2@Y_.transpose()@Y_)

@pymanopt.function.numpy(manifold_)
def euclidean_gradient(Y_):
    grad = 2*Q2@Y_.transpose()
    return grad.transpose()
                        
@pymanopt.function.numpy(manifold_)
def euclidean_hessian(Y_, U):
    hess = 2*Q2@U.transpose()
    return hess.transpose()

problem = Problem(manifold=manifold_, cost=cost,euclidean_gradient=euclidean_gradient, 
                  euclidean_hessian=euclidean_hessian)

optimizer = optimizers.TrustRegions(max_iterations=200)

Y_init_ = np.zeros((d,35))
h = manifold_.random_point()
Y_init_[:d,:] =  h#manifold_.retraction(Y_init,0.01*manifold_.random_tangent_vector(Y_init))#Y_init#h#manifold_.retraction(h,10*manifold_.random_tangent_vector(h))#Y_init
start = time.time()
Y_star = optimizer.run(problem,initial_point=Y_init_).point
end = time.time()
end-start

(14, 35)
Optimizing...
                                            f: +3.047438e+06   |grad|: 9.273354e+05
acc       k:     1     num_inner:     0     f: +1.146333e+06   |grad|: 7.888703e+05   negative curvature
acc       k:     2     num_inner:     0     f: +4.388036e+05   |grad|: 6.193354e+05   exceeded trust region
acc       k:     3     num_inner:     2     f: +1.111803e+05   |grad|: 2.270740e+05   exceeded trust region
acc TR+   k:     4     num_inner:     5     f: +2.799515e+04   |grad|: 6.540073e+04   exceeded trust region
acc TR-   k:     5     num_inner:    14     f: +2.050052e+04   |grad|: 8.587153e+04   exceeded trust region
acc       k:     6     num_inner:     8     f: +1.230574e+03   |grad|: 9.724771e+03   reached target residual-kappa (linear)
acc TR+   k:     7     num_inner:    25     f: +1.448857e+02   |grad|: 5.474109e+03   exceeded trust region
acc       k:     8     num_inner:     9     f: +8.037745e+01   |grad|: 3.801359e+02   reached target residual-kappa (linear

REJ TR-   k:    69     num_inner:   816     f: +8.428742e-08   |grad|: 1.495606e-04   exceeded trust region
acc TR+   k:    70     num_inner:   188     f: +6.599157e-08   |grad|: 4.697966e-02   exceeded trust region
acc       k:    71     num_inner:    18     f: +5.505262e-08   |grad|: 1.014811e-03   reached target residual-theta (superlinear)
acc       k:    72     num_inner:   192     f: +4.567482e-08   |grad|: 1.222743e-01   exceeded trust region
acc       k:    73     num_inner:     9     f: +2.326396e-08   |grad|: 8.067340e-03   reached target residual-kappa (linear)
acc       k:    74     num_inner:    57     f: +1.973649e-08   |grad|: 2.495992e-05   reached target residual-theta (superlinear)
REJ TR-   k:    75     num_inner:   186     f: +1.973649e-08   |grad|: 2.495992e-05   exceeded trust region
acc TR+   k:    76     num_inner:   173     f: +1.605895e-08   |grad|: 1.366313e-02   exceeded trust region
acc       k:    77     num_inner:    27     f: +1.470744e-08   |grad|: 1.27

2.256624698638916

In [469]:
U,S,V = np.linalg.svd(Y_star,full_matrices=False)
S

array([3.51198486e+00, 2.54417113e+00, 2.42069971e+00, 2.10696779e+00,
       1.59939892e+00, 1.35801395e+00, 1.04722418e+00, 6.27152310e-01,
       4.21647145e-02, 3.06672035e-04, 1.38169765e-07, 7.54878678e-08,
       2.04080028e-08, 2.23954606e-09, 6.86736125e-10, 5.42221902e-10,
       1.77954284e-10, 7.69323607e-11, 6.20638613e-11, 2.95902954e-11,
       1.81057718e-11, 1.14792761e-11, 6.60323299e-12, 5.07920808e-12,
       3.24170191e-12, 2.38783647e-12, 8.37121294e-13, 2.68794364e-13,
       6.09199745e-14, 2.33584254e-14])

In [426]:
cost(manifold_.retraction(Y_init,0.001*manifold_.random_tangent_vector(Y_init)))

0.16949905020091527

In [457]:
R@X[-1,:]


array([6.29999973e+02, 2.06917531e-02, 5.85000030e+02])

In [450]:
from scipy.spatial.transform import Rotation as R

v = np.array([ 4.99999990e+01, -1.75732342e-02, -7.92915769e-03])

R = rotation_matrix_from_vectors(v/np.linalg.norm(v),e1/np.linalg.norm(e1))

In [110]:
v = R@Y_star[:,4] 

R1 = rotation_matrix_from_vectors(v/np.linalg.norm(v),-e2/10)

In [56]:
def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
    return rotation_matrix

In [444]:
X = np.linalg.pinv(C@C.transpose())@C@D@Y_star.transpose()
X

array([[-197.91080786,   -3.51634056, -334.24204664],
       [-247.89323014,   46.5012295 , -334.23588802],
       [-247.90287387,   -3.4970035 , -284.23412062],
       [-247.9108069 ,   -3.49876732, -334.23411748],
       [-222.91288883,  -53.49980509, -334.23632927],
       [-222.90873947,   46.50019549, -334.2398568 ],
       [-222.84423135,  -53.48005276,  225.76366645],
       [-222.80782815,   46.51994059,  225.76013394],
       [ 242.18191181,   -3.64845953,  250.61310708],
       [ 292.1818991 ,   -3.66666162,  250.59740557],
       [ 292.16367654,  -53.66668447,  250.62389412],
       [ 292.20011417,   46.33324046,  250.70951177],
       [ 332.18190682,   -3.68131659,  250.68864201],
       [ 382.18189813,   -3.6995146 ,  250.66599788]])

In [446]:
X = - X + X[3,:]
X

array([[ 4.99999990e+01, -1.75732342e-02, -7.92915769e-03],
       [ 1.75767593e-02,  4.99999968e+01, -1.77053346e-03],
       [ 7.93303407e-03,  1.76382543e-03,  4.99999969e+01],
       [-0.00000000e+00, -0.00000000e+00, -0.00000000e+00],
       [ 2.49979181e+01, -5.00010378e+01, -2.21178570e-03],
       [ 2.50020674e+01,  4.99989628e+01, -5.73931117e-03],
       [ 2.50665755e+01, -4.99812854e+01,  5.59997784e+02],
       [ 2.51029787e+01,  5.00187079e+01,  5.59994251e+02],
       [ 4.90092719e+02, -1.49692210e-01,  5.84847225e+02],
       [ 5.40092706e+02, -1.67894296e-01,  5.84831523e+02],
       [ 5.40074483e+02, -5.01679171e+01,  5.84858012e+02],
       [ 5.40110921e+02,  4.98320078e+01,  5.84943629e+02],
       [ 5.80092714e+02, -1.82549267e-01,  5.84922759e+02],
       [ 6.30092705e+02, -2.00747281e-01,  5.84900115e+02]])

In [233]:
X[-1,:]-position

array([-1.07039074e+01, -4.44089210e-16, -6.63956294e-01])

In [67]:
R@X[-1,:]

array([620.0773281 ,  72.70829173, 987.47895214])

In [52]:
G.edges

EdgeView([('x', 'y'), ('x', 'a1_origin'), ('x', 'a1_plus'), ('y', 'a1_origin'), ('y', 'a1_plus'), ('a1_origin', 'a1_plus'), ('a1_origin', 'a2_plus'), ('a1_origin', ' a2_minus'), ('a1_plus', 'a2_plus'), ('a1_plus', ' a2_minus'), ('a2_plus', ' a2_minus'), ('a2_plus', 'a3_plus'), ('a2_plus', 'a3_minus'), (' a2_minus', 'a3_plus'), (' a2_minus', 'a3_minus'), ('a3_plus', 'a3_minus'), ('a3_plus', 'a4_origin'), ('a3_plus', 'a4_minus'), ('a3_minus', 'a4_origin'), ('a3_minus', 'a4_minus'), ('a4_origin', 'a4_minus'), ('a4_origin', 'a5_plus'), ('a4_origin', 'a5_minus'), ('a4_minus', 'a5_plus'), ('a4_minus', 'a5_minus'), ('a5_plus', 'a5_minus'), ('a5_plus', 'a6_origin'), ('a5_plus', 'a6_minus'), ('a5_minus', 'a6_origin'), ('a5_minus', 'a6_minus'), ('a6_origin', 'a6_minus')])